In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation, Add
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
# Define the input shape based on your dataset's image dimensions
input_shape = (48, 48, 1)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import os
import zipfile
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [3]:
dataset_path = "/kaggle/input/fer2013-cleaned-dataset/images1"
batch_size = 64

In [4]:
from keras.models import Sequential
import numpy as np
import tensorflow as tf
np.random.seed(42)
tf.random.set_seed(42)

In [5]:
from keras.layers import Layer, Input, Conv2D, BatchNormalization, Activation, Add, MaxPooling2D, Flatten, Dense
from keras.models import Model

class SelfAttention(Layer):
    def __init__(self, channels):
        super(SelfAttention, self).__init__()
        self.channels = channels

        # Query, Key, and Value transformations
        self.W_q = Conv2D(channels // 8, (1, 1), padding='same')
        self.W_k = Conv2D(channels // 8, (1, 1), padding='same')
        self.W_v = Conv2D(channels, (1, 1), padding='same')

    def call(self, x):
        q = self.W_q(x)
        k = self.W_k(x)
        v = self.W_v(x)

        # Reshape for compatibility with matrix multiplication
        q = tf.reshape(q, [-1, tf.shape(q)[1] * tf.shape(q)[2], self.channels // 8])
        k = tf.reshape(k, [-1, tf.shape(k)[1] * tf.shape(k)[2], self.channels // 8])
        v = tf.reshape(v, [-1, tf.shape(v)[1] * tf.shape(v)[2], self.channels])

        # Attention weights
        attention_weights = tf.nn.softmax(tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(tf.cast(tf.shape(k)[-1], tf.float32)))

        # Weighted sum
        output = tf.matmul(attention_weights, v)

        # Reshape back to the original spatial dimensions
        output = tf.reshape(output, [-1, tf.shape(x)[1], tf.shape(x)[2], self.channels])

        return output

def residual_block_with_attention(x, filters, kernel_size=(3, 3), stride=(1, 1), padding='same'):
    shortcut = x

    # First convolution layer
    x = Conv2D(filters, kernel_size, strides=stride, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Self-Attention layer
    x_att = SelfAttention(filters)(x)

    # Combine original features and attention features
    x = Add()([x, x_att])

    # Second convolution layer
    x = Conv2D(filters, kernel_size, padding=padding)(x)
    x = BatchNormalization()(x)

    # Add the shortcut to the output
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

# Build the model with self-attention
def build_vgg_with_resnet_and_attention(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # VGG-like convolutional layers with residual blocks and self-attention
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(input_layer)
    x = residual_block_with_attention(x, 64)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = residual_block_with_attention(x, 128)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = residual_block_with_attention(x, 256)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = residual_block_with_attention(x, 512)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Flatten and add fully connected layers
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(512, activation='relu')(x)

    # Output layer
    output_layer = Dense(num_classes, activation='softmax')(x)

    # Create and compile the model
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

# Define the number of classes in your dataset
num_classes = 7  # Assuming FER 2013 has 7 emotion classes

# Build the model with self-attention
model = build_vgg_with_resnet_and_attention(input_shape, num_classes)

# Compile the model with appropriate loss and optimizer

# Print a summary of the model architecture
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 48, 48, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 48, 48, 64)           640       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 48, 48, 64)           36928     ['conv2d[0][0]']              
                                                                                                  
 batch_normalization (Batch  (None, 48, 48, 64)           256       ['conv2d_1[0][0]']            
 Normalization)                                                                               

In [6]:
train_data_generator = ImageDataGenerator(rescale=1./255)
test_data_generator = ImageDataGenerator(rescale=1./255)

train_data = train_data_generator.flow_from_directory(
    os.path.join(dataset_path, 'train'),
    color_mode='grayscale',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_data = test_data_generator.flow_from_directory(
    os.path.join(dataset_path, 'test'),
    color_mode='grayscale',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 28044 images belonging to 7 classes.
Found 7177 images belonging to 7 classes.


In [7]:
num_train_samples = 28044
num_test_samples = 7177

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [14]:
history = model.fit(
    train_data,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=100,
    callbacks=[early_stopping],
    batch_size=batch_size,
    validation_data=test_data
)

Epoch 1/100
438/438 [==============================] - 73s 167ms/step - loss: 0.6969 - accuracy: 0.7390 - val_loss: 1.2763 - val_accuracy: 0.5548
Epoch 2/100
438/438 [==============================] - 74s 168ms/step - loss: 0.6676 - accuracy: 0.7504 - val_loss: 1.3279 - val_accuracy: 0.5734
Epoch 3/100
438/438 [==============================] - 74s 168ms/step - loss: 0.5977 - accuracy: 0.7778 - val_loss: 1.3539 - val_accuracy: 0.5682
Epoch 4/100
438/438 [==============================] - 74s 170ms/step - loss: 0.5798 - accuracy: 0.7835 - val_loss: 1.5085 - val_accuracy: 0.5526
Epoch 5/100
438/438 [==============================] - 74s 169ms/step - loss: 0.5216 - accuracy: 0.8040 - val_loss: 1.4284 - val_accuracy: 0.5763
Epoch 6/100
438/438 [==============================] - 74s 169ms/step - loss: 0.4507 - accuracy: 0.8352 - val_loss: 1.5143 - val_accuracy: 0.5754


In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_true = test_data.classes  # Assuming you're using flow_from_directory
predictions = model.predict(test_data)


y_pred = np.argmax(predictions, axis=1)


accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')



print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

113/113 [==============================] - 7s 54ms/step
Accuracy: 0.5754493520969765
Precision: 0.5747998300032895
Recall: 0.5754493520969765
F1 Score: 0.5744061574469045


In [17]:
from sklearn.metrics import classification_report, roc_auc_score

# Assuming you have trained your model and evaluated it
# test_data, num_test_samples, batch_size, model

# Make predictions on test data
predictions = model.predict(test_data, steps=num_test_samples // batch_size)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_data.classes[:len(predicted_classes)]  # Extract true classes from the test_data

# Calculate AUC score
auc = roc_auc_score(test_data.labels[:len(predicted_classes)], predictions, multi_class='ovr') 

# Generate classification report
report = classification_report(true_classes, predicted_classes)
print("Classification Report:\n", report)


112/112 [==============================] - 7s 58ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.44      0.47       957
           1       0.47      0.41      0.44       111
           2       0.40      0.38      0.39      1024
           3       0.78      0.81      0.80      1774
           4       0.50      0.54      0.52      1233
           5       0.45      0.48      0.46      1247
           6       0.75      0.69      0.72       822

    accuracy                           0.58      7168
   macro avg       0.55      0.54      0.54      7168
weighted avg       0.57      0.58      0.57      7168

